# ***Prediction models design***


## *Данный, второй документ в цикле статей описывает процесс выбора и обучения ML-модели для прогнозирования исходов теннисных матчей*


## Skills changelog:
*это первая попытка использования Machine Learning как инструмента в принципе, и поэтому
- есть только общее понимание того, что из себя представляет Data Science в целом и Machine Learning в частности;
- есть огромное желание попробовать ML на практике;
- используются любые руководства для начинающих;
- отсутствуют специфические знания и навыки, а именно - для подготовки данных, обучения моделей и т.п.;
- все, чем приходится пока руководствоваться - логика и здравый смысл.


## Sources:
- Python 3 and Pandas docs;
- StackOverflow


feature selection test

In [ ]:
### !pip install pandas
### !pip install matplotlib
import pandas as pd
import numpy as np
#import urllib
import urllib.request
import os
import glob
import csv
import re
from datetime import datetime
from IPython.display import display
from pandas.plotting import scatter_matrix
import matplotlib

# Pandas display options tweaking
pd.options.display.max_columns = None
INIT_MAX_ROWS = 50
pd.options.display.max_rows = INIT_MAX_ROWS

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '../../'))
print('Root directory:', ROOT_DIR)
# Downloaded files path
DOWNL_F_P = os.path.join(ROOT_DIR, 'match_data', 'match_data_downloaded')
# Preprocessed files path
PREP_F_P = os.path.join(ROOT_DIR, 'match_data', 'match_data_preprocessed')

In [ ]:
preprocessed_data_file = 'atp_matches_1991-2018_preprocessed_final.csv'
preprocessed_data_full_path = os.path.join(PREP_F_P, preprocessed_data_file)
preprocessed_data = pd.read_csv(preprocessed_data_full_path, header=0, encoding='utf-8-sig', engine='python')

display(preprocessed_data)

In [ ]:
%time scatter_matrix(preprocessed_data, figsize=(400,400))